In [1]:
import math
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter



In [3]:
!pip install streamlit newspaper3k nltk networkx wordcloud newspaper3k pysentimiento textblob

import streamlit as st
from newspaper import Article
import re
import math
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import newspaper
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from pysentimiento import create_analyzer
import pandas as pd
import json
from textblob import TextBlob

st.set_page_config(page_title="NLP", page_icon="📰")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 83.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 37.9 

In [4]:
!pip install langdetect

from langdetect import detect


import threading
_lock = threading.Lock()


def display_title(text):
    print("# " + text)

def display_text(text):
    print(text)

display_title("📰Article Processor")
display_text("Enter the URLs of Articles to process:")
display_text(" News Article, Wiki, etc")


urls = []
for i in range(2):
    url = input("URL {}: ".format(i+1))
    urls.append(url)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=b32cad3f1df5c53b0a6f82324e943161e5f5df4a5e9629c4a13c0183abb859a9
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
# 📰Article Processor
Enter the URLs of Articles to process:
 News Article, Wiki, etc
URL 1: https://thehackernews.com/
URL 2: https://www.livemint.com/news/trends/passwords-leaked-hackers-post-file-with-1000-crore-passwords-online-in-biggest-cyber-security-breach-credential-stuffing-11720598530725.html


In [5]:

def text_input(label, visibility, disabled, placeholder):
    print(label)
    print("Placeholder:", placeholder)
    print("Visibility:", visibility)
    print("Disabled:", disabled)
    value = input("Enter URL: ")
    return value


user_urls = []
visibility = "visible"
disabled = False
placeholder = "e.g. https://TOInews.com/article/trending/delhi-elections/"


user_urls.append(text_input("URL 1", visibility, disabled, placeholder))


max_urls = 5


for i in range(1, max_urls):
    if i < len(user_urls):
        checkbox_value = input(f"URL {i + 1} (Checkbox Value): ")
        if checkbox_value.lower() == "true":
            user_urls.append(text_input(f"URL {i + 1}", visibility, disabled, placeholder))



URL 1
Placeholder: e.g. https://TOInews.com/article/trending/delhi-elections/
Visibility: visible
Disabled: False
Enter URL: https://www.livemint.com/news/trends/passwords-leaked-hackers-post-file-with-1000-crore-passwords-online-in-biggest-cyber-security-breach-credential-stuffing-11720598530725.html


In [6]:
!pip install newspaper3k langdetect nltk

import numpy as np
from newspaper import Article
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from langdetect import detect

# Ensure you have the NLTK stopwords corpus downloaded
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent_tokenize(sent1)]
    sent2 = [w.lower() for w in sent_tokenize(sent2)]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def create_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for index1 in range(len(sentences)):
        for index2 in range(len(sentences)):
            if index1 == index2:
                continue
            similarity_matrix[index1][index2] = sentence_similarity(sentences[index1], sentences[index2], stop_words)

    return similarity_matrix

def process_article(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    title = article.title
    text = article.text
    summary = article.summary
    keywords = article.keywords
    language = detect(text)

    sentences = sent_tokenize(text)
    stop_words = stopwords.words('english')
    similarity_matrix = create_similarity_matrix(sentences, stop_words)

    summary2 = text_summary(text)  # Placeholder function for a custom summary

    news_item = {
        "Title": title,
        "Article Text": text,
        "Article Summary": summary,
        "Summary 2.0": summary2,
        "Article Keywords": keywords,
        "Language": language,
        "Similarity Matrix": similarity_matrix.tolist()  # Convert to list for JSON serialization
    }

    return news_item

def process_urls(user_urls):
    user_news_data = []

    for url in user_urls:
        try:
            news_item = process_article(url)
            user_news_data.append(news_item)
        except Exception as e:
            print(f"Error processing URL: {url}. Error: {e}")

    return user_news_data

def text_summary(text):
    # Placeholder for an actual text summary function
    return "This is a summary of the text."

user_urls = [
    "https://thehackernews.com/",
    "https://cybernews.com/",
    "https://cyware.com/cyber-security-news-articles"
]
user_news_data = process_urls(user_urls)

for news in user_news_data:
    print(news)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Error processing URL: https://cybernews.com/. Error: Article `download()` failed with 403 Client Error: Forbidden for url: https://cybernews.com/ on URL https://cybernews.com/
Error processing URL: https://cyware.com/cyber-security-news-articles. Error: Article `download()` failed with 403 Client Error: Forbidden for url: https://cyware.com/cyber-security-news-articles on URL https://cyware.com/cyber-security-news-articles
{'Title': 'The Hacker News | #1 Trusted Cybersecurity News Site', 'Article Text': 'Smash-and-Grab Extortion\n\nThe Problem The "2024 Attack Intelligence Report" from the staff at Rapid7 [1] is a well-researched, well-written report that is worthy of careful study. Some key takeaways are: 53% of the over 30 new vulnerabilities that were widely exploited in 2023 and at the start of 2024 were zero-days . More mass compromise events arose from zero-day vulnerabilities than from n-day vulnerabilities. Nearly a quarter of widespread attacks were zero-day attacks where a si

In [7]:
def text_summary(article_id, top_n=3):
    import networkx as nx
    from numpy import ndarray
    from collections import Counter

    with open("en-stop-words.txt", "r") as sw:
        stop_words = sw.read().split()

    summarize_text = []

    sentences = read_article_text(article_id)
    print("Sentences:", sentences)

    sentence_similarity_matrix = create_similarity_matrix(sentences, stop_words)

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    print("Scores:", scores)

    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    print("Ranked Sentence:", ranked_sentence)

    for i in range(min(top_n, len(ranked_sentence))):
        summarize_text.append(ranked_sentence[i][1])
    summarize_text = ". ".join(summarize_text)


    print("Summarized Text: \n", summarize_text)
    return summarize_text

In [8]:
!pip install newspaper3k langdetect

from newspaper import Article, build
from langdetect import detect

def text_summary(text):
    # Placeholder for an actual text summary function
    return "This is a summary of the text."

def process_urls(user_urls):
    user_news_data = []

    for url in user_urls:
        try:
            # Build a newspaper source from the URL
            paper = build(url, memoize_articles=False)

            for article in paper.articles:
                try:
                    article.download()
                    article.parse()

                    title = article.title
                    text = article.text
                    article.nlp()
                    summary = article.summary
                    keywords = article.keywords
                    language = detect(text)  # Detect the language

                    summary2 = text_summary(text)  # Replace with actual text_summary function

                    news_item = {
                        "Title": title,
                        "Article Text": text,
                        "Article Summary": summary,
                        "Summary 2.0": summary2,
                        "Article Keywords": keywords,
                        "Language": language
                    }

                    user_news_data.append(news_item)
                except Exception as e:
                    print(f"Error processing article from URL: {url}. Error: {e}")

        except Exception as e:
            print(f"Error processing URL: {url}. Please enter a valid URL. Only English Articles URL are Supported for now. Error: {e}")

    return user_news_data

user_urls = [
    "https://thehackernews.com/",
    "https://cybernews.com/",
    "https://cyware.com/cyber-security-news-articles"
]
user_news_data = process_urls(user_urls)

for news in user_news_data:
    print(news)


CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://thehackernews.com/feed
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://thehackernews.com/rss
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://thehackernews.com/feeds
CRITICAL:newspaper.extractors:Must extract urls from either html, text or doc!
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/feeds
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/feed
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/rss
CRITICAL:newspaper.extractors:Must extract urls from either html, text or doc!
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for ur

{'Title': 'Palo Alto Networks Patches Critical Flaw in Expedition Migration Tool', 'Article Text': 'Palo Alto Networks has released security updates to address five security flaws impacting its products, including a critical bug that could lead to an authentication bypass.\n\nCataloged as CVE-2024-5910 (CVSS score: 9.3), the vulnerability has been described as a case of missing authentication in its Expedition migration tool that could lead to an admin account takeover.\n\n"Missing authentication for a critical function in Palo Alto Networks Expedition can lead to an Expedition admin account takeover for attackers with network access to Expedition," the company said in an advisory. "Configuration secrets, credentials, and other data imported into Expedition is at risk due to this issue."\n\nThe flaw impacts all versions of Expedition prior to version 1.2.92, which remediates the problem. Synopsys Cybersecurity Research Center\'s (CyRC) Brian Hysell has been credited with discovering an

In [9]:
import json
from newspaper import Article, build
from langdetect import detect

def text_summary(text):
    # Placeholder for an actual text summary function
    return "This is a summary of the text."

def process_urls(user_urls):
    user_news_data = []

    for url in user_urls:
        try:
            # Build a newspaper source from the URL
            paper = build(url, memoize_articles=False)

            for article in paper.articles:
                try:
                    article.download()
                    article.parse()

                    title = article.title
                    text = article.text
                    article.nlp()
                    summary = article.summary
                    keywords = article.keywords
                    language = detect(text)  # Detect the language

                    summary2 = text_summary(text)  # Replace with actual text_summary function

                    news_item = {
                        "Title": title,
                        "Article Text": text,
                        "Article Summary": summary,
                        "Summary 2.0": summary2,
                        "Article Keywords": keywords,
                        "Language": language
                    }

                    user_news_data.append(news_item)
                except Exception as e:
                    print(f"Error processing article from URL: {url}. Error: {e}")

        except Exception as e:
            print(f"Error processing URL: {url}. Please enter a valid URL. Only English Articles URL are Supported for now. Error: {e}")

    # Convert list of news items to JSON
    return json.dumps(user_news_data, indent=4)

user_urls = [
    "https://thehackernews.com/",
    "https://cybernews.com/",
    "https://cyware.com/cyber-security-news-articles"
]

user_news_data_json = process_urls(user_urls)

print(user_news_data_json)


CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://thehackernews.com/feed
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://thehackernews.com/rss
CRITICAL:newspaper.network:[REQUEST FAILED] 404 Client Error: Not Found for url: https://thehackernews.com/feeds
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/rss
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/feeds
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cybernews.com/feed
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cyware.com/cyber-security-news-articles
CRITICAL:newspaper.network:[REQUEST FAILED] 403 Client Error: Forbidden for url: https://cyware.com/rss
CRI

[
    {
        "Title": "Palo Alto Networks Patches Critical Flaw in Expedition Migration Tool",
        "Article Text": "Palo Alto Networks has released security updates to address five security flaws impacting its products, including a critical bug that could lead to an authentication bypass.\n\nCataloged as CVE-2024-5910 (CVSS score: 9.3), the vulnerability has been described as a case of missing authentication in its Expedition migration tool that could lead to an admin account takeover.\n\n\"Missing authentication for a critical function in Palo Alto Networks Expedition can lead to an Expedition admin account takeover for attackers with network access to Expedition,\" the company said in an advisory. \"Configuration secrets, credentials, and other data imported into Expedition is at risk due to this issue.\"\n\nThe flaw impacts all versions of Expedition prior to version 1.2.92, which remediates the problem. Synopsys Cybersecurity Research Center's (CyRC) Brian Hysell has been cr